In [1]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm import tqdm
from glob import glob
from skimage.io import imsave
import tifffile

BASE_DIR = Path(r'F:\spatial_data_test\processed')
RUN_ID = '20230523_HCC_PRISM_probe_refined'
src_dir = BASE_DIR / f'{RUN_ID}_processed'
stc_dir = src_dir / 'stitched'
read_dir = src_dir / 'readout'
density_dir = read_dir / 'density'
os.makedirs(density_dir, exist_ok=True)

In [2]:
rna_df = pd.read_csv(read_dir/'mapped_genes.csv')[['Y','X','Gene']]
rna_df['Y'] = rna_df['Y'].astype(int)
rna_df['X'] = rna_df['X'].astype(int)
genes = rna_df['Gene'].unique()
rna_df.head()

,Y,X,Gene
0,2567,27994,PRISM_1
1,6484,24166,Other
2,8176,24412,Other
3,8256,24186,Other
4,6807,24142,Other


In [3]:
# read imshape using tifffile without loading the whole image
im_shape = tifffile.TiffFile(stc_dir/'cyc_1_cy3.tif').pages[0].shape

def plot_density_downsample(df, filename, fac=100):
    y = (im_shape[0] // fac) + 1
    x = (im_shape[1] // fac) + 1
    coordinates = df[['Y','X']].to_numpy()
    canvas = np.zeros((y*fac,x*fac),dtype=np.uint16)
    canvas[coordinates[:,0],coordinates[:,1]] = 1
    canvas_down = canvas.reshape(y,fac,x,fac).sum(-1).sum(1)
    imsave(filename,canvas_down.astype(np.uint16), check_contrast=False)


for gene in tqdm(genes):
    if glob(str(density_dir/f'{gene}.tif')): continue
    else: plot_density_downsample(rna_df[rna_df['Gene']==gene], density_dir/f'{gene}.tif')

  0%|          | 0/32 [00:00<?, ?it/s]

100%|██████████| 32/32 [00:52<00:00,  1.63s/it]
